In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


# Ignore warnings
# import warnings
# warnings.filterwarnings("ignore")

plt.ion()   # interactive mode




# training data
# training data loader

# testing data
# testing data loader

In [ ]:
directory = 'cs156b\cs156b_train_data_small' # change if necessary
csv_file = pd.read_csv('cs156b/train2023.csv')

df = csv_file

y_cols = ['No Finding', 'Enlarged Cardiomediastinum', 
            'Cardiomegaly', 'Lung Opacity', 'Pneumonia', 
            'Pleural Effusion', 'Pleural Other', 'Fracture', 
            'Support Devices']

# why are we doing this filling of NaN values to be 0?
csv_file[y_cols] = csv_file[y_cols].fillna(0)

# df.fillna(0, inplace=True)


Y = df[y_cols]

X = df["Path"]

print(X[:5],Y[:5])

In [ ]:
!  pip install imageio-ffmpeg

In [ ]:

# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
from PIL import Image
from torchvision.transforms import Compose, ToTensor, Resize

class CustomDataset(Dataset):
    """My custom chexpert dataset"""
    
    def __init__(self, csv_file, root_dir, transform=None):

        """
        Args:
            csv_file (string): Path to the CSV file with annotations.
            root_dir (string): Directory with all the images.
                defaults to /groups/CS156b/data
                where we have images in data/train and data/test
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.df = pd.read_csv(csv_file)
        # fil in NaN values with 0
        self.df[y_cols] = self.df[y_cols].fillna(0)
        
        # self.root_dir = root_dir
        # self.root_dir = "/groups/CS156b/data"
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

                   
        # I want to get a sample and the sample should be the Path that corresponds to the idx which is the first column
        row = self.df.iloc[idx]
        
        
        
        path = row['Path']
        path = path[6:]
        
        try:
            image_np = io.imread(os.path.join(self.root_dir, path), as_gray=True)
    
            # Apply transformations if needed
            if self.transform:
                image_pil = Image.fromarray(image_np)
                image_pil = self.transform(image_pil)
                image_np = np.array(image_pil)

            image = torch.tensor(image_np, dtype=torch.float32)

        except Exception as e:
            print("failed: ", e)
            # image is 224 x 224 in grayscale
            # to float tensor
            image = torch.zeros(1, 224, 224, dtype=torch.float32)
            
        # Convert labels to numpy array
        labels = row[y_cols].values
        
        labels = labels.astype(np.float32)


        # Convert numpy array to tensor, specifying the dtype as torch.float32
        labels = torch.tensor(labels, dtype=torch.float32)


            
        return image, labels




    

In [ ]:
from tqdm import tqdm

In [ ]:
# collect images that I have rn on my pc
# fixme this shouldn't matter for hpc but will need to remove code
idxs = []
for i in range(1, 501):
    # zfill for 5 digits
    stri = str(i).zfill(5)
    # print the idx of which "Path" has "pid{stri}"
    mask = df["Path"].fillna("").str.contains(f"pid{stri}")
    # print(f"pid{stri} is at index {df[mask].index}")
    for j in df[mask].index:
        idxs.append(j)
        
print(idxs)
len(idxs)

In [ ]:
import torch.nn as nn
import torchvision
from torchvision.models import resnet18
from torchvision.models.resnet import ResNet18_Weights, BasicBlock


In [ ]:

# customset = CustomDataset(csv_file='cs156b/train2023.csv', root_dir='cs156b/cs156b_train_data_small', )
from torchvision.transforms import Grayscale

# copilot
transformations = Compose([
    Grayscale(num_output_channels=1),
    Resize((224, 224)),
    ToTensor()
])

# Create the dataset with the transformations
customset = CustomDataset(csv_file='cs156b/train2023.csv', root_dir='cs156b/cs156b_train_data_small', transform=transformations)

subset = torch.utils.data.Subset(customset, idxs)



train_size = int(0.8 * len(subset))
test_size = len(subset) - train_size
# in the meantime, while I don't have the whole set
train_dataset, test_dataset = torch.utils.data.random_split(subset, [train_size, test_size])

# set batch size
batch_size = 32

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


train_features, train_labels = next(iter(train_data_loader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# print(train_labels)


In [ ]:
j = 0
for img, lab in subset:
    # show 10 imgs
    plt.imshow(img.squeeze(0), cmap='gray')
    plt.show()
    if j == 10:
        break

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import DataLoader
from tqdm import tqdm

# Load pre-trained ResNet
model = models.resnet18(pretrained=True)

# Use CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move the model to the CUDA device


# Change the first layer to accept grayscale images
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


# Change the number of output features of the last layer
num_classes = 9 
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the criterion to the CUDA device
model = model.to(device)
criterion = nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Assuming train_data_loader is your DataLoader for training data

losses = []

model.train()

for _ in tqdm(range(15)):
    for images, labels in train_data_loader:
        # Move images and labels to the CUDA device
        images = images.cuda()
        labels = labels.cuda()

        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        
        loss.backward()
        optimizer.step()

    print(f"Loss: {loss.item()}")
    losses.append(loss.item())


# plot loss
plt.plot(losses)

In [ ]:
# evaluate on test_data printing side by side

model.eval()

losses2 = []

for images, labels in test_data_loader:
    images = images.cuda()
    labels = labels.cuda()
    output = model(images)
    print("Predictions:")
    print(output)
    print("True labels:")
    print(labels)
    loss = criterion(output, labels)
    
    
    losses2.append(loss.item())
    
    
    

In [ ]:
import csv

test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()

# Open the CSV file
with open("predictions.csv", "w") as f:
    writer = csv.writer(f)

    for images, labels in test_data_loader:
        images = images.cuda()
        output = model(images)
        # Convert the predictions to a numpy array and write them to the CSV file
        writer.writerows(output.cpu().detach().numpy())